In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read the dataset into a pandas dataframe
dfyellow = pd.read_csv("rawdata/yellow_tripdata_2020-01.csv", \
            dtype={'PULocationID': int, 'total_amount': float}, \
            usecols=['PULocationID', 'total_amount'])

dfgreen = pd.read_csv("rawdata/green_tripdata_2020-01.csv", \
            dtype={'PULocationID': int, 'total_amount': float}, \
            usecols=['PULocationID', 'total_amount'])

In [3]:
dfyellow

,PULocationID,total_amount
0,238,11.27
1,239,12.30
2,238,10.80
3,238,8.16
4,193,4.80
...,...,...
6405003,237,21.14
6405004,259,62.46
6405005,137,51.90
6405006,50,30.22


In [4]:
dfzone = pd.read_csv('taxi_data/taxi+_zone_lookup.csv')

In [5]:
dfzone

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


LocationID 264 and 265 can be dropped as they also don't have a corresponding polygon in the shapefile

In [6]:
taxi_data = dfyellow.groupby(['PULocationID']).sum().reindex(range(1, len(dfzone)+1)).fillna(0)
green_taxi_data = dfgreen.groupby(['PULocationID']).sum().reindex(range(1, len(dfzone)+1)).fillna(0)

In [7]:
def myLog(x):
    return np.log(x) if (x > 0) else 0

#taxi_data['PULocationID'] = taxi_data.index
taxi_data.reset_index(level=0, inplace=True)
taxi_data["log_total_amount"] = np.array(list(map(myLog, taxi_data["total_amount"])))

green_taxi_data.reset_index(level=0, inplace=True)
green_taxi_data["log_total_amount"] = np.array(list(map(myLog, green_taxi_data["total_amount"])))

In [8]:
taxi_data["green_total_amount"] = green_taxi_data["total_amount"]
taxi_data["green_log_total_amount"] = green_taxi_data["log_total_amount"]
taxi_data.rename(columns={
                    "total_amount": "yellow_total_amount",
                    "log_total_amount": "yellow_log_total_amount"
                }, inplace=True)
taxi_data["Borough"] = dfzone["Borough"]
taxi_data["Zone"] = dfzone["Zone"]
taxi_data["service_zone"] = dfzone["service_zone"]

In [9]:
taxi_data

,PULocationID,yellow_total_amount,yellow_log_total_amount,green_total_amount,green_log_total_amount,Borough,Zone,service_zone
0,1,74478.65,11.218268,466.61,6.145494,EWR,Newark Airport,EWR
1,2,76.69,4.339771,0.00,0.000000,Queens,Jamaica Bay,Boro Zone
2,3,7942.37,8.979967,14820.08,9.603738,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,181770.74,12.110502,2972.79,7.997256,Manhattan,Alphabet City,Yellow Zone
4,5,2387.80,7.778128,0.00,0.000000,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...,...,...,...,...
260,261,793017.41,13.583600,2314.93,7.747135,Manhattan,World Trade Center,Yellow Zone
261,262,1372338.98,14.132027,1981.46,7.591589,Manhattan,Yorkville East,Yellow Zone
262,263,1876105.46,14.444709,10684.17,9.276518,Manhattan,Yorkville West,Yellow Zone
263,264,975087.21,13.790282,7538.27,8.927748,Unknown,NV,NaN


In [10]:
taxi_data.to_csv("taxi_data.csv", encoding='utf-8', index=False)